In [ ]:
from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator,
                                      DefaultScanOffsetsSettingsGenerator,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper)
from pipeline2.detection import LegacySpotPairFinder, SimpleSingleChannelSpotDetector
from pipeline2.imspector import ImspectorConnection
from pipeline2.stoppingcriteria import TimedStoppingCriterion

import specpy as sp

import numpy as np
from matplotlib import pyplot as plt


#BoundingBoxLocationGrouper()


In [ ]:
# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

# path to generic golbead config_path
gb_config_path = 'C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'

# imspector object
im = sp.Imspector()



pll = PipelineLevels('overview', 'detail', 'more_detail')


# overview task generator: same settings every time
atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                    DefaultLocationRemover(
                        JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/goldbeads_overview.json'])
                    )
                )
                .withDelay(.5))

# pipeline
pl = (AcquisitionPipeline('1')
        .withImspectorConnection(ImspectorConnection(im))
        .withPipelineLevels(pll)
        .withNameHandler(DefaultNameHandler('C:/Users//RESOLFT/Desktop/TEST_GEN/', pll))
        .withAddedStoppingCondition(TimedStoppingCriterion(200)))

# init detector
detector = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.overview), 2, 2).withPlotDetections().withVerbose(False)

# detail task generation: settings from last overview + new FOV + grouped detections
atg_detail = AcquisitionTaskGenerator(pll.detail,
                         NewestSettingsSelector(pl, pll.overview),
                         DefaultFOVSettingsGenerator([[1e-5, 1e-5, 0]], [[5e-8, 5e-8, 0]]),
                         DefaultScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector, [1e-5, 1e-5, 1e-5]).withVerbose()))

# overview callback: re-add overview
pl.withCallbackAtLevel(atg_overview, pll.overview)
# overview callback2: detect & do detail
pl.withCallbackAtLevel(atg_detail, pll.overview)


# another detector in detail
detector2 = SimpleSingleChannelSpotDetector(NewestDataSelector(pl, pll.detail), 5, 0.1, 0, 1.2).withPlotDetections().withVerbose()

# detail task generation: settings from last overview + new FOV + grouped detections
atg_detail_more = AcquisitionTaskGenerator(pll.more_detail,
                         NewestSettingsSelector(pl, pll.detail),
                         DefaultFOVSettingsGenerator([[1e-6, 1e-6, 0]], [[1e-8, 1e-8, 0]]),
                         DefaultScanOffsetsSettingsGenerator(BoundingBoxLocationGrouper(detector2, [1e-6, 1e-6, 1e-6]).withVerbose()))

# dummy callback: print & plot detections in detail img
#pl.withCallbackAtLevel(lambda x: print(detector2.get_locations()), pll.detail)

# overview callback2: detect & do detail
pl.withCallbackAtLevel(atg_detail_more, pll.detail)

# call overview atg once to add first measurement to pipeline
atg_overview(pl)

# GO
pl.run()


In [ ]:
im = sp.Imspector()
par = im.active_measurement().parameters('ExpControl/scan/range/z')
par

In [ ]:
from skimage.feature import peak_local_max

peak_local_max(np.array([[1,0,0,0], [0,0,0,1]]), min_distance=1)